# Instalacion de dependencias

In [2]:
#pip install spacy

In [3]:
#pip install happytransformer

In [4]:
#pip install nltk

In [5]:
#!python -m spacy download en_core_web_sm

In [6]:
#pip install wordcloud

In [7]:
#nltk.download('wordnet')

https://www.kaggle.com/datasets/nidhiy07/student-depression-text

https://www.kaggle.com/code/nidhiy07/forecast-depression-anxiety-text

1) Este conjunto de datos contiene información en formato Excel que comprende unos 7489 datos de redes sociales, comentarios de Facebook, etc.
2) Todas las personas seleccionadas para la anotación de datos dominan muy bien el inglés y son estudiantes, con edades comprendidas entre los 15 y los 17 años.
3)Hay 5 columnas en este conjunto de datos. Texto, etiquetas, edad, categoría de edad y sexo.
4) En las columnas de texto hay texto normal y de ansiedad/depresión, y la columna de etiquetas indica si el texto correspondiente denota ansiedad o depresión.
1 ANSIEDAD
0 NEUTRO

In [6]:
import pandas as pd
import numpy as np
import torch
from torch import nn, optim #Pytorch
from datasets import load_dataset

In [2]:
RANDOM_SEED = 42
BATCH_SIZE = 16
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


## Cargar datos

In [3]:
dir_pandas = 'C:/Users/anabe/Documents/PRUEBAS/DEPRESION ACT/corpus/{}'
df=pd.read_excel(dir_pandas.format('depresion.xlsx'))
#Solo para ver los primeros
df.head(10)

,text,label,Age,Gender,Age Category
0,let Gather For Party,0.0,16.0,Male,Teen Age
1,I hate being alive when I feel so dead inside.,1.0,17.0,Female,Teen Age
2,Exam are Comimg I m already Feeling Very Dep...,1.0,16.0,Male,Teen Age
3,Nice picture,0.0,16.0,Male,Teen Age
4,"Feeling worried, even though you actually have...",1.0,16.0,Male,Teen Age
5,"There are people who died, their friends didn'...",0.0,16.0,Male,Teen Age
6,keep it up hahaha,0.0,16.0,Male,Teen Age
7,The match cycle I had.Buy-brought-out-lost.,0.0,16.0,Male,Teen Age
8,"nctb! jewel case dream, which doesn't have a p...",0.0,16.0,Male,Teen Age
9,"college but it's like not going to college, bu...",0.0,16.0,Male,Teen Age


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7489 entries, 0 to 7488
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   text          7476 non-null   object 
 1   label         7486 non-null   float64
 2   Age           7486 non-null   float64
 3   Gender        7486 non-null   object 
 4   Age Category  7486 non-null   object 
dtypes: float64(2), object(3)
memory usage: 292.7+ KB


In [5]:
# Checking null values
df.isnull().sum()

text            13
label            3
Age              3
Gender           3
Age Category     3
dtype: int64

In [6]:
df.describe()

,label,Age
count,7486.000000,7486.000000
mean,0.163906,15.379108
std,0.370215,1.058104
min,0.000000,13.000000
25%,0.000000,14.000000
50%,0.000000,15.000000
75%,0.000000,16.000000
max,1.000000,17.000000


In [7]:
dup_df = df[df.duplicated()].shape[0]
print(f"There are {dup_df} duplicate entries among {df.shape[0]} entries in  dataset.")

There are 39 duplicate entries among 7489 entries in  dataset.


In [8]:
df = df.sample(frac = 1)

In [9]:
df['label'].value_counts().to_frame().T.style.set_properties(**{"background-color": "#F3FFFF","color":"black","border": "1.5px  solid black"})

label,0.000000,1.000000
count,6259,1227


In [10]:
df.head()

,text,label,Age,Gender,Age Category
2827,"It's okay, I also have no intention to be angr...",0.0,14.0,Male,Young Age
2858,COMO Q JYP RIPEO,0.0,14.0,Male,Young Age
2133,Pharmaceutical stocks are in ruins right now. ...,0.0,17.0,Male,Teen Age
4357,â€œSuicide it too hard to commitâ€Watch me. G...,1.0,16.0,Female,Teen Age
2908,It's not really nice to be ignored ðŸ˜©,0.0,14.0,Male,Young Age


In [11]:
df.dtypes

text             object
label           float64
Age             float64
Gender           object
Age Category     object
dtype: object

In [12]:
df['label'].value_counts()

label
0.0    6259
1.0    1227
Name: count, dtype: int64

In [13]:
not_used_cols = ['Age','Gender',"Age Category"]
df1 = df.drop(not_used_cols,axis=1)
df1.sample(3)

,text,label
5839,qn es yuri,0.0
4953,"In the previous project, my uname is different...",0.0
3581,"I'm so nervous I swear, the views are getting ...",1.0


In [14]:
df1 = df1.dropna(axis=0)
print("DataFrame after removing rows with NaN value in any column:")
print(df1)

DataFrame after removing rows with NaN value in any column:
                                                   text  label
2827  It's okay, I also have no intention to be angr...    0.0
2858                                   COMO Q JYP RIPEO    0.0
2133  Pharmaceutical stocks are in ruins right now. ...    0.0
4357  â€œSuicide it too hard to commitâ€Watch me. G...    1.0
2908            It's not really nice to be ignored ðŸ˜©    0.0
...                                                 ...    ...
5191  Looking for mutualan with fellow nctzens, who ...    0.0
5226                                 Ami ajke onk happy    0.0
5390                              como que jyp mur10 ee    0.0
860      Arya, if you play futsal, you don't use a goal    0.0
7270             I feel excited . Gosh, a little tremor    0.0

[7476 rows x 2 columns]


In [15]:
df1.shape

(7476, 2)

In [16]:
df1

,text,label
2827,"It's okay, I also have no intention to be angr...",0.0
2858,COMO Q JYP RIPEO,0.0
2133,Pharmaceutical stocks are in ruins right now. ...,0.0
4357,â€œSuicide it too hard to commitâ€Watch me. G...,1.0
2908,It's not really nice to be ignored ðŸ˜©,0.0
...,...,...
5191,"Looking for mutualan with fellow nctzens, who ...",0.0
5226,Ami ajke onk happy,0.0
5390,como que jyp mur10 ee,0.0
860,"Arya, if you play futsal, you don't use a goal",0.0


In [17]:
df1['label'].unique()

array([0., 1.])

In [18]:
df1

,text,label
2827,"It's okay, I also have no intention to be angr...",0.0
2858,COMO Q JYP RIPEO,0.0
2133,Pharmaceutical stocks are in ruins right now. ...,0.0
4357,â€œSuicide it too hard to commitâ€Watch me. G...,1.0
2908,It's not really nice to be ignored ðŸ˜©,0.0
...,...,...
5191,"Looking for mutualan with fellow nctzens, who ...",0.0
5226,Ami ajke onk happy,0.0
5390,como que jyp mur10 ee,0.0
860,"Arya, if you play futsal, you don't use a goal",0.0


In [19]:
df1.shape

(7476, 2)

In [20]:
df1.dtypes

text      object
label    float64
dtype: object

In [21]:
df1['label'] = df1['label'].astype('int64')
df1.dtypes

text     object
label     int64
dtype: object

In [22]:
df1['label'] = df1['label'].apply(lambda _: str(_))

In [23]:
df1

,text,label
2827,"It's okay, I also have no intention to be angr...",0
2858,COMO Q JYP RIPEO,0
2133,Pharmaceutical stocks are in ruins right now. ...,0
4357,â€œSuicide it too hard to commitâ€Watch me. G...,1
2908,It's not really nice to be ignored ðŸ˜©,0
...,...,...
5191,"Looking for mutualan with fellow nctzens, who ...",0
5226,Ami ajke onk happy,0
5390,como que jyp mur10 ee,0
860,"Arya, if you play futsal, you don't use a goal",0


In [24]:
df1.dtypes

text     object
label    object
dtype: object

In [25]:
from sklearn.model_selection import train_test_split 
train_data, train_test = train_test_split(df1, test_size = 0.20)

print("Ejemplos usados para entrenar: ", len(train_data))
print("Ejemplos usados para test: ", len(train_test))

Ejemplos usados para entrenar:  5980
Ejemplos usados para test:  1496


In [26]:
train_test.to_csv("C:/Users/anabe/Documents/PRUEBAS/OPENAI/DEPRESION/test4.csv", index=False)

In [27]:
def convert_to_gpt4_format(dataset):
    fine_tuning_data = []
    for _, row in dataset.iterrows():
        json_response = '{"label": "' + row['label'] + '"}'
        fine_tuning_data.append({
            "messages": [
                {"role": "user", "content": row['text']},
                {"role": "assistant", "content": json_response}
            ]
        })
    return fine_tuning_data



In [28]:
dataset = df1
converted_data = convert_to_gpt4_format(dataset)
converted_data[0]['messages']

[{'role': 'user',
  'content': "It's okay, I also have no intention to be angry about this"},
 {'role': 'assistant', 'content': '{"label": "0"}'}]

In [29]:
import json
json.loads(converted_data[0]['messages'][-1]['content'])

{'label': '0'}

In [30]:
from sklearn.model_selection import train_test_split

# Stratified splitting. Assuming 'Top Category' can be used for stratification
train_data, val_data = train_test_split(
    converted_data,
    test_size=0.2,
    stratify=dataset['label'],
    random_state=42  # for reproducibility
)

In [31]:
type(train_data[0])

dict

In [32]:
def write_to_jsonl(data, file_path):
    with open(file_path, 'w') as file:
        for entry in data:
            json.dump(entry, file)
            file.write('\n')


training_file_name = "train4.jsonl"
validation_file_name = "val4.jsonl"

write_to_jsonl(train_data, training_file_name)
write_to_jsonl(val_data, validation_file_name)

ENTRENAMIENTO


In [8]:
import os
os.environ['OPENAI_API_KEY'] = 'sk-******'

In [9]:
import openai

In [10]:
import openai
from packaging import version

required_version = version.parse("1.13.3") # replace the version by the version you want
current_version = version.parse(openai.__version__)

if current_version < required_version:
    raise ValueError(f"Error: OpenAI version {openai.__version__}"
                     " is less than the required version 1.1.1")
else:
    print("OpenAI version is compatible.")

# -- Now we can get to it
from openai import OpenAI

print('OPENAI WAS GREAT AGAIN')

OpenAI version is compatible.
OPENAI WAS GREAT AGAIN


In [11]:
import openai
print(openai.VERSION)

1.13.3


In [12]:
from openai import OpenAI
client = OpenAI(
  api_key=os.environ['OPENAI_API_KEY'],  # this is also the default, it can be omitted
)

In [38]:
training_file = client.files.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
validation_file = client.files.create(
    file=open(validation_file_name, "rb"), purpose="fine-tune"
)

print("Training file id:", training_file.id)
print("Validation file id:", validation_file.id)

Training file id: file-NbDzE6NwNKh714sEqYT1ZsA2
Validation file id: file-kJ7dc6UkL2fbdexPwPV285JS


In [39]:
#from openai import OpenAI
#client = OpenAI()
 #client.files.create(file=open('data/train.jsonl', "rb"), purpose="fine-tune")
#client.files.create(
 # file=open("C:/Users/anabe/Documents/PRUEBAS/OPENAI/DEPRESION/train.jsonl", "rb"),
 # purpose="fine-tune"
#)

In [40]:
suffix_name = "upm"

response = client.fine_tuning.jobs.create(
    training_file=training_file.id,
    validation_file=validation_file.id,
    model="gpt-4o-mini-2024-07-18",
    suffix=suffix_name,
    hyperparameters={
    "n_epochs":3}
)
response

FineTuningJob(id='ftjob-F03Xcq1cI2kGuEleqa34kdIY', created_at=1729442605, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-fmYIRHxHTOs4FIil8mL5yGRD', result_files=[], status='validating_files', trained_tokens=None, training_file='file-NbDzE6NwNKh714sEqYT1ZsA2', validation_file='file-kJ7dc6UkL2fbdexPwPV285JS', user_provided_suffix='upm', seed=183837913, estimated_finish=None, integrations=[])

https://github.com/PradipNichite/Youtube-Tutorials/blob/main/GPT3.5%20Finetuning/GPT_3_5_Finetuning_Tutorial.ipynb

In [41]:
client.fine_tuning.jobs.list(limit=1)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-F03Xcq1cI2kGuEleqa34kdIY', created_at=1729442605, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-fmYIRHxHTOs4FIil8mL5yGRD', result_files=[], status='validating_files', trained_tokens=None, training_file='file-NbDzE6NwNKh714sEqYT1ZsA2', validation_file='file-kJ7dc6UkL2fbdexPwPV285JS', user_provided_suffix='upm', seed=183837913, estimated_finish=None, integrations=[])], object='list', has_more=True)

In [42]:
response = client.fine_tuning.jobs.retrieve("ftjob-F03Xcq1cI2kGuEleqa34kdIY")
response
     

FineTuningJob(id='ftjob-F03Xcq1cI2kGuEleqa34kdIY', created_at=1729442605, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-fmYIRHxHTOs4FIil8mL5yGRD', result_files=[], status='validating_files', trained_tokens=None, training_file='file-NbDzE6NwNKh714sEqYT1ZsA2', validation_file='file-kJ7dc6UkL2fbdexPwPV285JS', user_provided_suffix='upm', seed=183837913, estimated_finish=None, integrations=[])

In [44]:
fine_tuned_model_id = response.fine_tuned_model
print("\nFine-tuned model id:", fine_tuned_model_id)


Fine-tuned model id: None


In [1]:
fine_tuned_model_id='ft:gpt-4o-mini-2024-07-18:espe:upm:AKU63gok'

https://www.youtube.com/watch?v=YioMpE2jGIU

### TEST

In [2]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def format_test(row):

    formatted_message = [
        {
            "role": "user",
            "content": row['text']
        }
    ]
    return formatted_message


def predict(test_messages, fine_tuned_model_id):

    response = client.chat.completions.create(
        model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=50
    )

    return response.choices[0].message.content
     

In [3]:
def store_predictions(test_df, fine_tuned_model_id):

    print("fine_tuned_model_id",fine_tuned_model_id)
    test_df['Prediction'] = None

    for index, row in test_df.iterrows():
        test_message = format_test(row)
        prediction_result = predict(test_message, fine_tuned_model_id)
        test_df.at[index, 'Prediction'] = prediction_result

        test_df.to_csv("C:/Users/anabe/Documents/PRUEBAS/OPENAI/DEPRESION/predictions4.csv")

In [13]:
test_df = pd.read_csv("test4.csv")
store_predictions(test_df, fine_tuned_model_id)

fine_tuned_model_id ft:gpt-4o-mini-2024-07-18:espe:upm:AKU63gok


https://scale.com/blog/fine-tune-gpt-3.5

In [ ]:
#pip install evals

In [5]:
#!oaieval gpt-3.5-turbo test-sentiment-stu

In [14]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import os

In [15]:
data = pd.read_csv('C:/Users/anabe/Documents/PRUEBAS/OPENAI/DEPRESION/predictions4.csv')

In [16]:
data.dtypes

Unnamed: 0     int64
text          object
label          int64
Prediction    object
dtype: object

In [17]:
data

,Unnamed: 0,text,label,Prediction
0,0,Long-term suffering. It's been two years eh..P...,0,"{""label"": ""0""}"
1,1,"Who is still in class today? Yes, me. ”âœ‹",0,"{""label"": ""0""}"
2,2,online / holiday?,0,"{""label"": ""0""}"
3,3,"Even when I was on school leave, I didn't ask,...",0,"{""label"": ""0""}"
4,4,Yeayy tomorrow is a day off from work,0,"{""label"": ""0""}"
...,...,...,...,...
1491,1491,I'm always depressed,1,"{""label"": ""1""}"
1492,1492,"Don't make decisions when you're not fit, let ...",1,"{""label"": ""1""}"
1493,1493,"Hot plus cool = sore throat, want to poop",0,"{""label"": ""0""}"
1494,1494,good morning worldðŸ˜»,0,"{""label"": ""0""}"


In [18]:
print(data['Prediction'])

0       {"label": "0"}
1       {"label": "0"}
2       {"label": "0"}
3       {"label": "0"}
4       {"label": "0"}
             ...      
1491    {"label": "1"}
1492    {"label": "1"}
1493    {"label": "0"}
1494    {"label": "0"}
1495    {"label": "0"}
Name: Prediction, Length: 1496, dtype: object


In [19]:
import pandas as pd
from ast import literal_eval

df = pd.read_csv('C:/Users/anabe/Documents/PRUEBAS/OPENAI/DEPRESION/predictions4.csv', converters={'Prediction': literal_eval})
df['labelx'] = df['Prediction'].apply(pd.Series)
df


,Unnamed: 0,text,label,Prediction,labelx
0,0,Long-term suffering. It's been two years eh..P...,0,{'label': '0'},0
1,1,"Who is still in class today? Yes, me. ”âœ‹",0,{'label': '0'},0
2,2,online / holiday?,0,{'label': '0'},0
3,3,"Even when I was on school leave, I didn't ask,...",0,{'label': '0'},0
4,4,Yeayy tomorrow is a day off from work,0,{'label': '0'},0
...,...,...,...,...,...
1491,1491,I'm always depressed,1,{'label': '1'},1
1492,1492,"Don't make decisions when you're not fit, let ...",1,{'label': '1'},1
1493,1493,"Hot plus cool = sore throat, want to poop",0,{'label': '0'},0
1494,1494,good morning worldðŸ˜»,0,{'label': '0'},0


In [20]:
df['labelx'] = df['labelx'].astype('int64')
df.dtypes

Unnamed: 0     int64
text          object
label          int64
Prediction    object
labelx         int64
dtype: object

In [21]:
df.to_csv("C:/Users/anabe/Documents/PRUEBAS/OPENAI/DEPRESION/predictions-final4.csv")

In [22]:
import pandas as pd
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import os
def evaluate_results(original, prediction, model_name):
    data = pd.read_csv('C:/Users/anabe/Documents/PRUEBAS/OPENAI/DEPRESION/predictions-final4.csv')
    accuracy = round(accuracy_score(data[original], data[prediction]), 4)
    precision = round(precision_score(data[original], data[prediction], average='weighted'), 4)
    recall = round(recall_score(data[original], data[prediction], average='weighted'), 4)
    f1 = round(f1_score(data[original], data[prediction], average='weighted'), 4)

    # Create a DataFrame with the evaluation results including the 'model' column
    evaluation_df = pd.DataFrame({
        'Model': [model_name],
        'Accuracy': [accuracy],
        'Precision': [precision],
        'Recall': [recall],
        'F1': [f1]
    })

    # Append the results to the existing CSV file or create a new one
    evaluation_df.to_csv('C:/Users/anabe/Documents/PRUEBAS/OPENAI/DEPRESION/evaluation-results4.csv', mode='a', header=not os.path.exists('C:/Users/anabe/Documents/PRUEBAS/OPENAI/DEPRESION/evaluation-results4.csv'), index=False)

    return {'Model': model_name, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1': f1}

In [23]:
print(f'base:gpt-4o-mini: ' + str(evaluate_results('label', 'labelx', 'base:gpt-4o-mini')))

base:gpt-4o-mini: {'Model': 'base:gpt-4o-mini', 'Accuracy': 0.9893, 'Precision': 0.9896, 'Recall': 0.9893, 'F1': 0.9894}


https://arize.com/blog-course/precision-vs-recall/